In [1]:
# dev of foolbox api attacks

In [2]:
from __future__ import print_function

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

from advertorch import attacks
from advertorch.utils import predict_from_logits
from advertorch_examples.utils import _imshow

In [5]:
!gpustat

countdown  Mon Aug 12 13:27:42 2019
[0] GeForce GTX 1080 | 39'C,   0 % |   565 /  8119 MB | felix(553M)
[1] GeForce GTX 1080 | 38'C,   0 % |  2811 /  8118 MB | felix(1975M)


In [6]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(0)

CUDA Available:  True
Using device: cuda


In [7]:
# # random seeds
# torch.manual_seed(999)
# np.random.seed(999)

# torch.backends.cudnn.deterministic = True
# torch.cuda.manual_seed_all(999)

In [8]:
# MNIST dataset and dataloader declaration
# transforms does both the conversion from 0-255 to 0-1
# and normalizes by the precomputed mean and std

batch_size = 192

mnist_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=True, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

mnist_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=False, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

In [9]:
import foolbox

In [10]:
# load fgn model
#  test one model
model_path = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth"
dict_path =  "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_state_dict.pth"
params_paths = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_parameters.txt"

print(model_path)
# exp params
with open(params_paths) as f: 
    print(f.read())

../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth
hidden_l_nums [32, 32]
drop_p 0.03125
ordinal 3.0
lambda for l2 loss 6.66666666667e-06
lambda for sigma loss 0.00666666666667
noisy_centers False
batch_size 192



In [11]:
# # full load
fgn_model = torch.load(model_path)
fgn_model.to(device)

# dict load
# fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, out_feats=10, hidden_l_nums=[64,64], 
#                                      drop_p=0, noisy_centers=False).to(device)
# fgn_model.load_state_dict(state_dict=torch.load(dict_path))

Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.03125)
    (1): FGN_layer()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.03125)
    (4): FGN_layer()
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.03125)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): FGN_layer()
)

In [12]:
# test the model
### Loss Functions for the FGN
fgn_cross_ent_loss = (lambda model,output,target:  F.cross_entropy(output,target.long()))

fgn_test_res = fgnh.test(fgn_model, mnist_train_loader, fgn_cross_ent_loss,
                               pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2914, Accuracy: 54272/60000 (90%)


In [13]:
# set random eval to false to check 
fgn_model.set_random_eval(False)
fgn_test_res = fgnh.test(fgn_model, mnist_train_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

fgn_test_res = fgnh.test(fgn_model, mnist_test_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2918, Accuracy: 54331/60000 (91%)
Test set - Average loss: 0.3059, Accuracy: 9040/10000 (90%)


In [14]:
# change to random eval
fgn_model.set_random_eval(True)
fgn_test_res = fgnh.test(fgn_model, mnist_train_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

fgn_test_res = fgnh.test(fgn_model, mnist_test_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2913, Accuracy: 54273/60000 (90%)
Test set - Average loss: 0.3050, Accuracy: 9034/10000 (90%)


In [15]:
# create the attack
fmodel = foolbox.models.PyTorchModel(
    fgn_model, bounds=(-0.4242, 2.8214867), num_classes=10)

In [22]:
# get first images of dataset batch, post transform
for cln_data, labels in mnist_train_loader:
    break
    
image, label = cln_data[0], labels[0]

In [23]:
# minimum/maximum pixel value post normalization
min_pix = np.min(cln_data.cpu().numpy())
print("min_pix", min_pix)
max_pix = np.max(cln_data.cpu().numpy())
print("max_pix", max_pix)
# theoritical minimum
print( (0.0/255-0.1307)/0.3081 )
# theoritical maximum
print( (255.0/255-0.1307)/0.3081 )

min_pix -0.42421296
max_pix 2.8214867
-0.424212917884
2.82148653035


In [29]:
print('label', label)
print('predicted class', np.argmax(fmodel.forward_one(image)))

label tensor(5)


AttributeError: 'PyTorchModel' object has no attribute 'forward_one'

In [30]:
fmodel.predictions(image=image)

TypeError: data type not understood

In [31]:
foolbox.__version__

'1.3.2'

In [59]:
# instantiate the model
resnet18 = models.resnet18(pretrained=True).eval()
if torch.cuda.is_available():
    resnet18 = resnet18.cuda()
mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
fmodel = foolbox.models.PyTorchModel(
    resnet18, bounds=(0, 1), num_classes=1000, preprocessing=(mean, std))

# get source image and label
image, label = foolbox.utils.imagenet_example(data_format='channels_first')
image = image / 255.  # because our model expects values in [0, 1]

print('label', label)
print('predicted class', np.argmax(fmodel.predictions(image)))

# apply attack on source image
attack = foolbox.attacks.FGSM(fmodel)
adversarial = attack(image, label)

print('adversarial class', np.argmax(fmodel.predictions(adversarial)))

label 282
predicted class 282
adversarial class 281


In [60]:
image

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)